# Adapter

Construtor que adapta uma interface existente X para adaptar-se a interface requerida Y

Imagine que você tenha a seguinte API, que desenha apenas o ponto.

In [2]:
def draw_point(p):
    print('.', end='')        

Porém você está trabalhando com o código a seguir:

In [3]:
class Point:
    def __init__(self, x, y):
        self.y = y
        self.x = x
        
class Line:
    def __init__(self, start, end):
        self.end = end
        self.start = start
        
class Rectangle(list):
    '''Representa uma lista de linhas'''
    
    def __init__(self, x, y, width, hegith):
        self.append(Line(Point(x, y), Point(x + width, y)))
        self.append(Line(Point(x, y), Point(x, y + width)))
        self.append(Line(Point(x + width, y), Point(x, y+ width)))
        self.append(Line(Point(x, y + width), Point(x + width, y+ width)))

Como fazer para utilizar a função de desenho disponibilizada (draw_point), para desenhar o retângulo

In [4]:
class LineToPointAdapter(list):
    count = 0
    
    def __init__(self,line):
        self.count += 1
        print(f'{self.count}: Generating points for line '
              f'[{line.start.x},{line.start.y}]→'
              f'[{line.end.x},{line.end.y}]')
        
        left = min(line.start.x, line.end.x)
        right = max(line.start.x, line.end.x)
        top = min(line.start.y, line.end.y)
        bottom = max(line.start.y, line.end.y)
        
        if right == left:
            for y in range(top,bottom):
                self.append(Point(left,y))
        if top == bottom:
            for x in range(left, right):
                self.append(Point(x,top))

In [5]:
def draw(rcs):
    print("\n\n--- Drawing some stuff ---\n")
    for rc in rcs:
        for line in rc:
            adapter = LineToPointAdapter(line)
            for p in adapter:
                draw_point(p)
            print('\n')

In [6]:
if __name__ == '__main__':
    rs = [
        Rectangle(1, 1, 10, 10),
        Rectangle(3, 3, 6, 6)
    ]
    draw(rs)
    draw(rs)



--- Drawing some stuff ---

1: Generating points for line [1,1]→[11,1]
..........

1: Generating points for line [1,1]→[1,11]
..........

1: Generating points for line [11,1]→[1,11]


1: Generating points for line [1,11]→[11,11]
..........

1: Generating points for line [3,3]→[9,3]
......

1: Generating points for line [3,3]→[3,9]
......

1: Generating points for line [9,3]→[3,9]


1: Generating points for line [3,9]→[9,9]
......



--- Drawing some stuff ---

1: Generating points for line [1,1]→[11,1]
..........

1: Generating points for line [1,1]→[1,11]
..........

1: Generating points for line [11,1]→[1,11]


1: Generating points for line [1,11]→[11,11]
..........

1: Generating points for line [3,3]→[9,3]
......

1: Generating points for line [3,3]→[3,9]
......

1: Generating points for line [9,3]→[3,9]


1: Generating points for line [3,9]→[9,9]
......



Para evitar gerar várias vezes as mesmas informações, podemos utilizar um cache para armazenar o que já foi gerado através do adaptador

In [7]:
class LineToPointAdapterCache(list):
    count = 0
    cache = {}
    
    def __init__(self,line):
        self.h = hash(line)
        if self.h in self.cache:
            return
        
        self.count += 1
        self.count += 1
        print(f'{self.count}: Generating points for line '
              f'[{line.start.x},{line.start.y}]→'
              f'[{line.end.x},{line.end.y}]')
        
        left = min(line.start.x, line.end.x)
        right = max(line.start.x, line.end.x)
        top = min(line.start.y, line.end.y)
        bottom = max(line.start.y, line.end.y)
        
        points = []
        
        if right == left:
            for y in range(top,bottom):
                points.append(Point(left,y))
        if top == bottom:
            for x in range(left, right):
                points.append(Point(x,top))
                
        self.cache[self.h] = points
        
    def __iter__(self):
        return iter(self.cache[self.h])

In [9]:
def draw(rcs):
    print("\n\n--- Drawing some stuff ---\n")
    for rc in rcs:
        for line in rc:
            adapter = LineToPointAdapterCache(line)
            for p in adapter:
                draw_point(p)
            print('\n')

In [10]:
if __name__ == '__main__':
    rs = [
        Rectangle(1, 1, 10, 10),
        Rectangle(3, 3, 6, 6)
    ]
    draw(rs)
    draw(rs)



--- Drawing some stuff ---

2: Generating points for line [1,1]→[11,1]
..........

2: Generating points for line [1,1]→[1,11]
..........

2: Generating points for line [11,1]→[1,11]


2: Generating points for line [1,11]→[11,11]
..........

2: Generating points for line [3,3]→[9,3]
......

2: Generating points for line [3,3]→[3,9]
......

2: Generating points for line [9,3]→[3,9]


2: Generating points for line [3,9]→[9,9]
......



--- Drawing some stuff ---

..........

..........



..........

......

......



......

